In [ ]:
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
%run Attention.py

In [ ]:
Qc_Dataset = torch.load('Qc_3680_2070.pt')
Qc_Dataset.shape

In [ ]:
n_test = 2070
n_freq_training = 400   # number of frequencies will be used for training
n_gap = 40           # number of frequencies between training and testing datasets
input_dim=25     # attention mechanism input dimension
num_neurons=256  # number of neurons to be used in the NN
n_test = 2070     # testing dataset size
num_layers= 5

# features (we can't change these)
n_train = Qc_Dataset.shape[0]-n_test
n_feature = 4  # number of features describing the ring resonator 
nf = Qc_Dataset.shape[1]-n_feature         # number of Qc values for each device
fmax = 500       # max sim. frequency THz
df = 0.5         # sim frequeny difference

# end of variables
output_dim=nf-n_freq_training-n_gap
T_length=n_feature+n_freq_training//input_dim

In [ ]:
Qctrain = torch.narrow(Qc_Dataset, 0, 0, n_train)
Qctest = torch.narrow(Qc_Dataset, 0, n_train, n_test)

In [ ]:
# Splitting into train and test datasets:
rnn_train_data, null_data = torch.utils.data.random_split(RNN_Dataset(dataset=Qctrain,n=n_freq_training,input_dim=input_dim, ngap = n_gap), (Qctrain.shape[0], 0)) 
null_data, rnn_test_data = torch.utils.data.random_split(RNN_Dataset(dataset=Qctest,n=n_freq_training,input_dim=input_dim, ngap = n_gap), (0, Qctest.shape[0])) 

In [ ]:
# Loading the data through DataLoader:

train_loader = DataLoader(rnn_train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(rnn_test_data, batch_size=32, shuffle=False)

In [ ]:
# Defining an RNN model

#del rnn_model
rnn_model = recurrent_model(T_length=T_length, input_dim=input_dim,  num_layers=num_layers, num_neurons=num_neurons, output_dim=output_dim, model_type='RNN', bidirectional=False)

eta = 0.001
loss_func = nn.MSELoss()
optimizer = torch.optim.AdamW(rnn_model.parameters(), lr=eta)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.5)

In [ ]:
# Training the RNN model
start = time.time()
rnn_results = train_network_reg(rnn_model, loss_func, train_loader, test_loader=test_loader, epochs=100, score_funcs={'R^2 score': r2_score}, 
                                device=device, optimizer=optimizer, lr_schedule=scheduler)
stop = time.time()
print('Processing time: %s sec' %(stop-start))

In [ ]:
# Presenting the performance of the RNN model
rnn_results

In [ ]:
# Plotting the accuracy of the fully connected model:
sns.lineplot(x='epoch', y='test R^2 score', data=rnn_results, label='rnn')
plt.title('Test R^2 Score of the RNN-Model')
plt.gcf().set_size_inches(9, 5.5)
plt.show()